## 1. Setup e Imports

In [1]:
import tensorflow as tf
import numpy as np
import pandas as pd
import os
import time
import matplotlib.pyplot as plt

print(f"TensorFlow version: {tf.__version__}")

# Crear directorio temporal para ejemplos
TEMP_DIR = "/tmp/tf_data_files"
os.makedirs(TEMP_DIR, exist_ok=True)
print(f"Directorio temporal: {TEMP_DIR}")

2025-11-17 15:51:58.741808: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 AVX_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-11-17 15:51:58.785992: I tensorflow/core/util/util.cc:169] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.


TensorFlow version: 2.10.0
Directorio temporal: /tmp/tf_data_files


## 2. Datasets en Memoria: from_tensors vs from_tensor_slices

### 2.1 Diferencia fundamental

In [2]:
# Crear datos de ejemplo
data = tf.constant([[1, 2], [3, 4], [5, 6]])
print(f"Datos originales:\n{data}\n")

# Opción 1: from_tensors - TODO el tensor como UN SOLO elemento
ds1 = tf.data.Dataset.from_tensors(data)
print("=== from_tensors() ===")
print(f"Número de elementos: {len(list(ds1))}")
for element in ds1:
    print(f"Elemento shape: {element.shape}")
    print(f"Contenido:\n{element.numpy()}\n")

# Opción 2: from_tensor_slices - Cada FILA es un elemento
ds2 = tf.data.Dataset.from_tensor_slices(data)
print("=== from_tensor_slices() ===")
print(f"Número de elementos: {len(list(ds2))}")
for i, element in enumerate(ds2):
    print(f"Elemento {i}: shape={element.shape}, valor={element.numpy()}")

Datos originales:
[[1 2]
 [3 4]
 [5 6]]

=== from_tensors() ===
Número de elementos: 1
Elemento shape: (3, 2)
Contenido:
[[1 2]
 [3 4]
 [5 6]]

=== from_tensor_slices() ===
Número de elementos: 3
Elemento 0: shape=(2,), valor=[1 2]
Elemento 1: shape=(2,), valor=[3 4]
Elemento 2: shape=(2,), valor=[5 6]


2025-11-17 15:52:01.102675: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 AVX_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


### 2.2 Caso de uso típico: Features y Labels

In [3]:
# Datos simulados: features y labels
features = np.array([[1.0, 2.0], [3.0, 4.0], [5.0, 6.0], [7.0, 8.0]])
labels = np.array([0, 1, 0, 1])

print(f"Features shape: {features.shape}")
print(f"Labels shape: {labels.shape}\n")

# Crear dataset con pares (feature, label)
dataset = tf.data.Dataset.from_tensor_slices((features, labels))

print("Dataset creado con pares (feature, label):")
for x, y in dataset:
    print(f"  Feature: {x.numpy()}, Label: {y.numpy()}")

Features shape: (4, 2)
Labels shape: (4,)

Dataset creado con pares (feature, label):
  Feature: [1. 2.], Label: 0
  Feature: [3. 4.], Label: 1
  Feature: [5. 6.], Label: 0
  Feature: [7. 8.], Label: 1


## 3. Trabajar con Archivos CSV usando TextLineDataset

### 3.1 Crear archivos CSV de ejemplo

In [4]:
# Crear un archivo CSV simple
csv_file = os.path.join(TEMP_DIR, "data.csv")

with open(csv_file, 'w') as f:
    f.write("feature1,feature2,feature3,label\n")  # Header
    for i in range(100):
        f1 = np.random.rand()
        f2 = np.random.rand()
        f3 = np.random.rand()
        label = np.random.randint(0, 2)
        f.write(f"{f1:.4f},{f2:.4f},{f3:.4f},{label}\n")

print(f"Archivo CSV creado: {csv_file}")
print(f"\nPrimeras 5 líneas:")
with open(csv_file, 'r') as f:
    for i, line in enumerate(f):
        if i < 5:
            print(f"  {line.strip()}")

Archivo CSV creado: /tmp/tf_data_files/data.csv

Primeras 5 líneas:
  feature1,feature2,feature3,label
  0.8407,0.2453,0.7529,0
  0.0266,0.4850,0.4117,0
  0.6381,0.4080,0.7292,0
  0.7227,0.4348,0.8482,0


### 3.2 Leer CSV con TextLineDataset

In [5]:
# Leer archivo línea por línea
dataset = tf.data.TextLineDataset(csv_file)

print("Dataset desde archivo CSV:")
print(f"Tipo: {dataset}\n")

# Ver primeras líneas (raw)
print("Primeras 3 líneas (sin procesar):")
for i, line in enumerate(dataset.take(3)):
    print(f"  {i}: {line.numpy().decode('utf-8')}")

Dataset desde archivo CSV:
Tipo: <TextLineDatasetV2 element_spec=TensorSpec(shape=(), dtype=tf.string, name=None)>

Primeras 3 líneas (sin procesar):
  0: feature1,feature2,feature3,label
  1: 0.8407,0.2453,0.7529,0
  2: 0.0266,0.4850,0.4117,0


### 3.3 Parsear CSV con map()

Transformamos cada línea de texto en tensores numéricos.

In [6]:
def parse_csv_line(line):
    """
    Parsea una línea CSV y retorna (features, label)
    """
    # Definir tipos de datos para cada columna
    record_defaults = [0.0, 0.0, 0.0, 0]  # 3 floats + 1 int
    
    # Decodificar CSV
    parsed = tf.io.decode_csv(line, record_defaults=record_defaults)
    
    # Separar features y label
    features = tf.stack(parsed[:-1])  # Primeras 3 columnas
    label = parsed[-1]                 # Última columna
    
    return features, label

# Saltar header y parsear
dataset_parsed = dataset.skip(1).map(parse_csv_line)

print("Dataset parseado (features, label):")
for i, (features, label) in enumerate(dataset_parsed.take(5)):
    print(f"  Ejemplo {i}: features={features.numpy()}, label={label.numpy()}")

Dataset parseado (features, label):
  Ejemplo 0: features=[0.8407 0.2453 0.7529], label=0
  Ejemplo 1: features=[0.0266 0.485  0.4117], label=0
  Ejemplo 2: features=[0.6381 0.408  0.7292], label=0
  Ejemplo 3: features=[0.7227 0.4348 0.8482], label=0
  Ejemplo 4: features=[0.9876 0.406  0.3044], label=1


### 3.4 Pipeline completa para entrenamiento

In [7]:
def create_csv_dataset(filepath, batch_size=32, shuffle_buffer=1000, is_training=True):
    """
    Crea un dataset optimizado desde un archivo CSV
    
    - shuffle: SOLO para training (importante!)
    - batch: agrupa ejemplos
    - prefetch: optimiza rendimiento
    """
    dataset = tf.data.TextLineDataset(filepath)
    dataset = dataset.skip(1)  # Saltar header
    dataset = dataset.map(parse_csv_line, num_parallel_calls=tf.data.AUTOTUNE)
    
    # Shuffle SOLO en training
    if is_training:
        dataset = dataset.shuffle(buffer_size=shuffle_buffer)
    
    dataset = dataset.batch(batch_size)
    dataset = dataset.prefetch(buffer_size=tf.data.AUTOTUNE)
    
    return dataset

# Crear datasets
train_dataset = create_csv_dataset(csv_file, batch_size=16, is_training=True)
val_dataset = create_csv_dataset(csv_file, batch_size=16, is_training=False)

print("Pipeline de training creada")
print(f"Dataset: {train_dataset}\n")

# Ver un batch
for features_batch, labels_batch in train_dataset.take(1):
    print(f"Batch de features shape: {features_batch.shape}")
    print(f"Batch de labels shape: {labels_batch.shape}")
    print(f"\nPrimeros 3 ejemplos del batch:")
    for i in range(3):
        print(f"  {i}: features={features_batch[i].numpy()}, label={labels_batch[i].numpy()}")

Pipeline de training creada
Dataset: <PrefetchDataset element_spec=(TensorSpec(shape=(None, 3), dtype=tf.float32, name=None), TensorSpec(shape=(None,), dtype=tf.int32, name=None))>

Batch de features shape: (16, 3)
Batch de labels shape: (16,)

Primeros 3 ejemplos del batch:
  0: features=[0.4411 0.3412 0.0292], label=0
  1: features=[0.2682 0.9693 0.317 ], label=0
  2: features=[0.7472 0.2661 0.9841], label=1


## 4. Datasets Sharded (Múltiples Archivos)

### 4.1 Crear múltiples archivos CSV

In [8]:
# Crear 5 archivos sharded
num_shards = 5
samples_per_shard = 200

shard_files = []
for shard_idx in range(num_shards):
    filename = os.path.join(TEMP_DIR, f"data-{shard_idx:04d}.csv")
    shard_files.append(filename)
    
    with open(filename, 'w') as f:
        f.write("feature1,feature2,feature3,label\n")
        for i in range(samples_per_shard):
            f1 = np.random.rand()
            f2 = np.random.rand()
            f3 = np.random.rand()
            label = np.random.randint(0, 2)
            f.write(f"{f1:.4f},{f2:.4f},{f3:.4f},{label}\n")

print(f"Archivos sharded creados:")
for f in shard_files:
    print(f"  {f}")

print(f"\nTotal samples: {num_shards * samples_per_shard}")

Archivos sharded creados:
  /tmp/tf_data_files/data-0000.csv
  /tmp/tf_data_files/data-0001.csv
  /tmp/tf_data_files/data-0002.csv
  /tmp/tf_data_files/data-0003.csv
  /tmp/tf_data_files/data-0004.csv

Total samples: 1000


### 4.2 Método 1: Leer con list_files y flat_map

In [9]:
# Listar archivos usando patrón
file_pattern = os.path.join(TEMP_DIR, "data-*.csv")
files_dataset = tf.data.Dataset.list_files(file_pattern, shuffle=False)

print("Archivos encontrados:")
for i, filepath in enumerate(files_dataset):
    print(f"  {i}: {filepath.numpy().decode('utf-8')}")

Archivos encontrados:
  0: /tmp/tf_data_files/data-0000.csv
  1: /tmp/tf_data_files/data-0001.csv
  2: /tmp/tf_data_files/data-0002.csv
  3: /tmp/tf_data_files/data-0003.csv
  4: /tmp/tf_data_files/data-0004.csv


### 4.3 flat_map: De archivos a líneas

**Diferencia clave:**
- `map()`: 1 entrada → 1 salida (archivo → archivo procesado)
- `flat_map()`: 1 entrada → MUCHAS salidas (1 archivo → muchas líneas)

In [10]:
def read_csv_file(filepath):
    """
    Lee un archivo CSV y retorna un dataset de líneas
    """
    dataset = tf.data.TextLineDataset(filepath)
    dataset = dataset.skip(1)  # Saltar header
    return dataset

# Usar flat_map para expandir archivos a líneas
lines_dataset = files_dataset.flat_map(read_csv_file)

print(f"Dataset de líneas desde múltiples archivos:")
print(f"Tipo: {lines_dataset}\n")

# Contar total de líneas
total_lines = sum(1 for _ in lines_dataset)
print(f"Total de líneas: {total_lines}")

# Ver ejemplos
print(f"\nPrimeras 5 líneas:")
for i, line in enumerate(lines_dataset.take(5)):
    print(f"  {i}: {line.numpy().decode('utf-8')}")

Dataset de líneas desde múltiples archivos:
Tipo: <FlatMapDataset element_spec=TensorSpec(shape=(), dtype=tf.string, name=None)>

Total de líneas: 1000

Primeras 5 líneas:
  0: 0.9327,0.1193,0.6504,0
  1: 0.7062,0.6836,0.9363,1
  2: 0.3119,0.6076,0.6110,0
  3: 0.9029,0.6125,0.0891,0
  4: 0.2686,0.3941,0.6769,1


### 4.4 Pipeline completa para sharded datasets

In [11]:
def create_sharded_dataset(file_pattern, batch_size=32, is_training=True):
    """
    Crea dataset desde múltiples archivos sharded
    
    Pipeline: list_files → flat_map → map(parse) → shuffle → batch → prefetch
    """
    # 1. Listar archivos
    files = tf.data.Dataset.list_files(file_pattern, shuffle=is_training)
    
    # 2. Leer todos los archivos (flat_map: archivo → muchas líneas)
    dataset = files.flat_map(
        lambda filepath: tf.data.TextLineDataset(filepath).skip(1)
    )
    
    # 3. Parsear líneas (map: línea → (features, label))
    dataset = dataset.map(parse_csv_line, num_parallel_calls=tf.data.AUTOTUNE)
    
    # 4. Shuffle solo en training
    if is_training:
        dataset = dataset.shuffle(buffer_size=1000)
    
    # 5. Batch y prefetch
    dataset = dataset.batch(batch_size)
    dataset = dataset.prefetch(buffer_size=tf.data.AUTOTUNE)
    
    return dataset

# Crear dataset desde archivos sharded
sharded_train_dataset = create_sharded_dataset(
    file_pattern, 
    batch_size=32, 
    is_training=True
)

print("Dataset sharded creado:")
print(f"{sharded_train_dataset}\n")

# Ver un batch
for features_batch, labels_batch in sharded_train_dataset.take(1):
    print(f"Batch shape: {features_batch.shape}")
    print(f"Labels shape: {labels_batch.shape}")
    print(f"Primeras 5 labels: {labels_batch.numpy()[:5]}")

Dataset sharded creado:
<PrefetchDataset element_spec=(TensorSpec(shape=(None, 3), dtype=tf.float32, name=None), TensorSpec(shape=(None,), dtype=tf.int32, name=None))>

Batch shape: (32, 3)
Labels shape: (32,)
Primeras 5 labels: [0 1 0 1 0]


## 5. Comparación: map() vs flat_map()

Visualicemos la diferencia con un ejemplo simple.

In [12]:
# Dataset de ejemplo: números
numbers = tf.data.Dataset.from_tensor_slices([1, 2, 3])

# map(): 1 entrada → 1 salida
print("=== map() - duplica cada número ===")
mapped = numbers.map(lambda x: x * 2)
for val in mapped:
    print(f"  {val.numpy()}")

# flat_map(): 1 entrada → MUCHAS salidas
print("\n=== flat_map() - crea rango desde 0 hasta el número ===")
# Convertir a int64 para evitar error de tipo
flat_mapped = numbers.flat_map(lambda x: tf.data.Dataset.range(tf.cast(x, tf.int64)))
for val in flat_mapped:
    print(f"  {val.numpy()}")

print("\nObserva: flat_map expande 1 elemento en MÚLTIPLES elementos")

=== map() - duplica cada número ===
  2
  4
  6

=== flat_map() - crea rango desde 0 hasta el número ===
  0
  0
  1
  0
  1
  2

Observa: flat_map expande 1 elemento en MÚLTIPLES elementos


## 6. Importancia de Prefetch

### 6.1 Comparación de rendimiento

In [49]:
def simulate_training_step(features, labels):
    """
    Simula un paso de entrenamiento que toma tiempo
    """
    time.sleep(0.01)  # Simular procesamiento
    return tf.reduce_mean(features)

def measure_pipeline_performance(dataset, num_batches=50, name="Dataset"):
    """
    Mide el tiempo de iteración sobre un dataset
    """
    start_time = time.time()
    
    for i, (features, labels) in enumerate(dataset):
        if i >= num_batches:
            break
        _ = simulate_training_step(features, labels)
    
    elapsed = time.time() - start_time
    print(f"{name}: {elapsed:.3f} segundos ({num_batches} batches)")
    return elapsed

print("Comparando rendimiento...\n")

# Recrear datos para esta comparación (evitar conflictos con variables anteriores)
test_features = np.array([[1.0, 2.0], [3.0, 4.0], [5.0, 6.0], [7.0, 8.0]])
test_labels = np.array([0, 1, 0, 1])

# Dataset SIN prefetch
dataset_no_prefetch = tf.data.Dataset.from_tensor_slices((test_features, test_labels))
dataset_no_prefetch = dataset_no_prefetch.batch(4)

# Dataset CON prefetch
dataset_with_prefetch = tf.data.Dataset.from_tensor_slices((test_features, test_labels))
dataset_with_prefetch = dataset_with_prefetch.batch(4)
dataset_with_prefetch = dataset_with_prefetch.prefetch(tf.data.AUTOTUNE)

time_no_prefetch = measure_pipeline_performance(dataset_no_prefetch, num_batches=20, name="Sin prefetch")
time_with_prefetch = measure_pipeline_performance(dataset_with_prefetch, num_batches=20, name="Con prefetch")

improvement = ((time_no_prefetch - time_with_prefetch) / time_no_prefetch) * 100
print(f"\nMejora con prefetch: {improvement:.1f}%")

Comparando rendimiento...

Sin prefetch: 0.014 segundos (20 batches)
Con prefetch: 0.016 segundos (20 batches)

Mejora con prefetch: -9.1%


### 6.2 Visualización del concepto de prefetch

**Sin prefetch:**
```
CPU: [preparar batch 1] -----> [preparar batch 2] -----> [preparar batch 3]
GPU:                    [usar batch 1]       [usar batch 2]       [usar batch 3]
                        ⬆️ GPU espera        ⬆️ GPU espera        ⬆️ GPU espera
```

**Con prefetch:**
```
CPU: [preparar batch 1] [preparar batch 2] [preparar batch 3]
GPU:                    [usar batch 1]     [usar batch 2]     [usar batch 3]
                        ⬆️ ¡Ya está listo! ⬆️ ¡Ya está listo! ⬆️ ¡Ya está listo!
```

## 7. Entrenar un modelo con el dataset completo

In [13]:
# Crear modelo simple
model = tf.keras.Sequential([
    tf.keras.layers.Dense(16, activation='relu', input_shape=(3,)),
    tf.keras.layers.Dense(8, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

model.compile(
    optimizer='adam',
    loss='binary_crossentropy',
    metrics=['accuracy']
)

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 16)                64        
                                                                 
 dense_1 (Dense)             (None, 8)                 136       
                                                                 
 dense_2 (Dense)             (None, 1)                 9         
                                                                 
Total params: 209
Trainable params: 209
Non-trainable params: 0
_________________________________________________________________
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 16)                64        
                                                                 
 dense_1 (Dense)             (None, 8)                 136

In [14]:
# Entrenar con dataset sharded
print("Entrenando modelo con dataset sharded...\n")

history = model.fit(
    sharded_train_dataset,
    epochs=3,
    verbose=1
)

print("\nEntrenamiento completado")

Entrenando modelo con dataset sharded...

Epoch 1/3
32/32 [==============================] - 1s 6ms/step - loss: 0.7168 - accuracy: 0.4930
Epoch 2/3
32/32 [==============================] - 1s 6ms/step - loss: 0.7168 - accuracy: 0.4930
Epoch 2/3
32/32 [==============================] - 0s 1ms/step - loss: 0.7020 - accuracy: 0.4650
Epoch 3/3
32/32 [==============================] - 0s 1ms/step - loss: 0.7020 - accuracy: 0.4650
Epoch 3/3
32/32 [==============================] - 0s 1ms/step - loss: 0.6972 - accuracy: 0.4950

Entrenamiento completado
32/32 [==============================] - 0s 1ms/step - loss: 0.6972 - accuracy: 0.4950

Entrenamiento completado


## 8. Best Practices: Pipeline completa recomendada

In [15]:
def create_production_pipeline(
    file_pattern,
    parse_fn,
    batch_size=32,
    shuffle_buffer=10000,
    is_training=True,
    cache=False
):
    """
    Pipeline de producción siguiendo mejores prácticas
    
    Orden recomendado:
    1. list_files (con shuffle opcional)
    2. flat_map o interleave para leer archivos
    3. cache (opcional, si dataset cabe en memoria)
    4. map con num_parallel_calls
    5. shuffle (solo training)
    6. repeat (para training continuo)
    7. batch
    8. prefetch
    """
    # 1. Listar archivos
    files = tf.data.Dataset.list_files(file_pattern, shuffle=is_training)
    
    # 2. Leer archivos en paralelo con interleave (mejor que flat_map)
    dataset = files.interleave(
        lambda filepath: tf.data.TextLineDataset(filepath).skip(1),
        cycle_length=4,  # Leer 4 archivos simultáneamente
        num_parallel_calls=tf.data.AUTOTUNE
    )
    
    # 3. Cache (opcional)
    if cache:
        dataset = dataset.cache()
    
    # 4. Parsear con paralelización
    dataset = dataset.map(parse_fn, num_parallel_calls=tf.data.AUTOTUNE)
    
    # 5. Shuffle solo en training
    if is_training:
        dataset = dataset.shuffle(buffer_size=shuffle_buffer)
    
    # 6. Repeat para training continuo
    if is_training:
        dataset = dataset.repeat()
    
    # 7. Batch
    dataset = dataset.batch(batch_size)
    
    # 8. Prefetch
    dataset = dataset.prefetch(buffer_size=tf.data.AUTOTUNE)
    
    return dataset

# Crear pipeline de producción
production_dataset = create_production_pipeline(
    file_pattern=file_pattern,
    parse_fn=parse_csv_line,
    batch_size=32,
    is_training=True,
    cache=False
)

print("Pipeline de producción creada:")
print(production_dataset)

Pipeline de producción creada:
<PrefetchDataset element_spec=(TensorSpec(shape=(None, 3), dtype=tf.float32, name=None), TensorSpec(shape=(None,), dtype=tf.int32, name=None))>


## 9. Resumen de Conceptos Clave

In [16]:
print("""
╔═══════════════════════════════════════════════════════════════════╗
║                    RESUMEN DE CONCEPTOS                           ║
╠═══════════════════════════════════════════════════════════════════╣
║                                                                   ║
║ 1. DATASETS EN MEMORIA                                            ║
║    • from_tensors()        → TODO como 1 elemento                 ║
║    • from_tensor_slices()  → Cada fila = 1 elemento              ║
║                                                                   ║
║ 2. LEER ARCHIVOS                                                  ║
║    • TextLineDataset       → Leer línea por línea                ║
║    • list_files             → Encontrar múltiples archivos        ║
║                                                                   ║
║ 3. TRANSFORMACIONES                                               ║
║    • map()                  → 1 entrada → 1 salida                ║
║    • flat_map()             → 1 entrada → MUCHAS salidas          ║
║    • interleave()           → flat_map + paralelización           ║
║                                                                   ║
║ 4. OPTIMIZACIONES                                                 ║
║    • shuffle()              → Solo en training!                   ║
║    • batch()                → Agrupar ejemplos                    ║
║    • prefetch()             → CPU/GPU en paralelo                 ║
║    • cache()                → Guardar en memoria                  ║
║    • num_parallel_calls     → AUTOTUNE para paralelizar           ║
║                                                                   ║
║ 5. ORDEN RECOMENDADO                                              ║
║    list_files → flat_map → cache → map → shuffle → repeat →     ║
║    → batch → prefetch                                             ║
║                                                                   ║
╚═══════════════════════════════════════════════════════════════════╝
""")


╔═══════════════════════════════════════════════════════════════════╗
║                    RESUMEN DE CONCEPTOS                           ║
╠═══════════════════════════════════════════════════════════════════╣
║                                                                   ║
║ 1. DATASETS EN MEMORIA                                            ║
║    • from_tensors()        → TODO como 1 elemento                 ║
║    • from_tensor_slices()  → Cada fila = 1 elemento              ║
║                                                                   ║
║ 2. LEER ARCHIVOS                                                  ║
║    • TextLineDataset       → Leer línea por línea                ║
║    • list_files             → Encontrar múltiples archivos        ║
║                                                                   ║
║ 3. TRANSFORMACIONES                                               ║
║    • map()                  → 1 entrada → 1 salida                ║
║    • flat_map()    

## 10. Cleanup - Limpiar archivos temporales

In [18]:
import shutil

if os.path.exists(TEMP_DIR):
    shutil.rmtree(TEMP_DIR)
    print(f"Directorio temporal eliminado: {TEMP_DIR}")

## Conclusiones

Este notebook cubrió:

1. **Diferencia entre `from_tensors()` y `from_tensor_slices()`**
2. **Lectura de archivos CSV con `TextLineDataset`**
3. **Parseo de datos con `map()` y `tf.io.decode_csv()`**
4. **Manejo de datasets sharded con `flat_map()` e `interleave()`**
5. **Diferencia entre `map()` y `flat_map()`**
6. **Importancia de `prefetch()` para rendimiento**
7. **Pipeline de producción completa**
8. **Best practices: orden recomendado de operaciones**

### Key Takeaways:

- `from_tensor_slices()` es el método principal para crear datasets
- `TextLineDataset` lee archivos sin cargar todo en memoria
- `flat_map()` expande 1 elemento en múltiples (archivos → líneas)
- `shuffle()` solo se aplica en training, nunca en validación/test
- `prefetch()` es esencial para mantener GPU ocupada
- El orden de operaciones importa para el rendimiento